In [2]:
%load_ext autoreload
%autoreload 2

from model import BaseModel
from dataset import MolerDataset, MolerData
from utils import pprint_pyg_obj
from torch_geometric.loader import DataLoader
import torch
from model_utils import get_params



dataset = MolerDataset(
    root = '/data/ongh0068', 
    raw_moler_trace_dataset_parent_folder = '/data/ongh0068/guacamol/trace_dir',
    output_pyg_trace_dataset_parent_folder = '/data/ongh0068/l1000/pyg_output_playground',
    split = 'train_0',
)

loader = DataLoader(dataset, batch_size=16, shuffle=False, follow_batch = [
    'correct_edge_choices',
    'correct_edge_types',
    'valid_edge_choices',
    'valid_attachment_point_choices',
    'correct_attachment_point_choice',
    'correct_node_type_choices',
    'original_graph_x',
    'correct_first_node_type_choices'
])
params = get_params(dataset)

for batch in loader:
#     batch.cuda()
    break

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
params

{'full_graph_encoder': {'input_feature_dim': 59,
  'atom_or_motif_vocab_size': 166},
 'partial_graph_encoder': {'input_feature_dim': 59,
  'atom_or_motif_vocab_size': 166},
 'mean_log_var_mlp': {'input_feature_dim': 832, 'output_size': 1024},
 'decoder': {'node_type_selector': {'input_feature_dim': 1344,
   'output_size': 167},
  'node_type_loss_weights': tensor([5.5000, 0.5500, 2.2489, 0.5500, 0.5500, 0.7156, 0.8666, 0.7145, 5.5000,
          1.8568, 2.1470, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000,
          5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000,
          5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000,
          5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000,
          5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000,
          5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000,
          5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 5.50

In [7]:
model = BaseModel(params, dataset)#.cuda()

In [11]:
ckpt_file_path = '/data/ongh0068/l1000/FYP-DrugDiscoveryWithDeepLearning/lightning_logs/version_4/checkpoints/epoch=99-step=1000.ckpt'

model = BaseModel.load_from_checkpoint(ckpt_file_path, params = params, dataset = dataset)

RuntimeError: Error(s) in loading state_dict for BaseModel:
	size mismatch for _full_graph_encoder._model._first_layer.weight: copying a param with shape torch.Size([4, 123, 64]) from checkpoint, the shape in current model is torch.Size([4, 96, 64]).
	size mismatch for _partial_graph_encoder._model._first_layer.weight: copying a param with shape torch.Size([4, 124, 64]) from checkpoint, the shape in current model is torch.Size([4, 97, 64]).

In [6]:
torch.load(ckpt_file_path)

{'epoch': 99,
 'global_step': 1000,
 'pytorch-lightning_version': '1.8.1',
 'state_dict': OrderedDict([('_full_graph_encoder.dummy_param',
               tensor([], device='cuda:0')),
              ('_full_graph_encoder._embed.weight',
               tensor([[-0.6925, -1.4216, -0.4632,  ..., -0.1275, -2.1861, -0.5675],
                       [ 2.4087, -0.6630, -0.7316,  ..., -0.1863, -1.2079,  0.8590],
                       [ 0.5250, -1.4956,  1.3491,  ...,  2.4941,  1.3546, -2.0407],
                       ...,
                       [ 0.2572, -1.1277,  0.4879,  ...,  0.2530, -0.0108,  1.7342],
                       [-0.2389,  0.5544, -1.6286,  ...,  0.4119, -1.4408,  0.9899],
                       [-1.9277, -0.9655, -0.7046,  ...,  1.1322,  0.1007,  0.1005]],
                      device='cuda:0')),
              ('_full_graph_encoder._model._first_layer.q',
               tensor([[ 0.2970],
                       [-0.2548],
                       [ 0.0567],
                      

In [4]:
with torch.no_grad():
    input_molecule_representations = model._full_graph_encoder(
        original_graph_node_categorical_features=batch.original_graph_node_categorical_features,
        node_features=batch.original_graph_x.float(),
        edge_index=batch.original_graph_edge_index,
        edge_type=batch.original_graph_edge_type.int(),
        batch_index=batch.original_graph_x_batch,
    )


    partial_graph_representions, node_representations = model._partial_graph_encoder(
        partial_graph_node_categorical_features = batch.partial_node_categorical_features,
        node_features = batch.x,
        edge_index = batch.edge_index.long(), 
        edge_type = batch.edge_type, 
        graph_to_focus_node_map = batch.focus_node,
        candidate_attachment_points = batch.valid_attachment_point_choices,
        batch_index = batch.batch
    )
    # Apply latent sampling strategy
    p, q, latent_representations = model.sample_from_latent_repr(
        input_molecule_representations
    )

In [6]:
dataset.num_node_types

166

In [5]:
decoder_states = model.decode(latent_representations = latent_representations)

RDKit runtime error on base molecule, with message:
unsupported operand type(s) for -: '_vectSt6vectorIiSaIiEE' and '_vectSt6vectorIiSaIiEE'
RDKit runtime error on base molecule, with message:
unsupported operand type(s) for -: '_vectSt6vectorIiSaIiEE' and '_vectSt6vectorIiSaIiEE'
RDKit runtime error on base molecule, with message:
unsupported operand type(s) for -: '_vectSt6vectorIiSaIiEE' and '_vectSt6vectorIiSaIiEE'
RDKit runtime error on base molecule, with message:
unsupported operand type(s) for -: '_vectSt6vectorIiSaIiEE' and '_vectSt6vectorIiSaIiEE'
RDKit runtime error on base molecule, with message:
unsupported operand type(s) for -: '_vectSt6vectorIiSaIiEE' and '_vectSt6vectorIiSaIiEE'
RDKit runtime error on base molecule, with message:
unsupported operand type(s) for -: '_vectSt6vectorIiSaIiEE' and '_vectSt6vectorIiSaIiEE'
RDKit runtime error on base molecule, with message:
unsupported operand type(s) for -: '_vectSt6vectorIiSaIiEE' and '_vectSt6vectorIiSaIiEE'
RDKit runtime

RuntimeError: index_select(): Expected dtype int32 or int64 for index

In [8]:
from rdkit.Chem import Draw
Draw.MolsToGridImage([decoder_states[i].molecule for i in range(len(decoder_states))], subImgSize=(1000,1000))

NameError: name 'decoder_states' is not defined

In [29]:
from rdkit import Chem
Chem.MolToSmiles(decoder_states[1].molecule)

'CCC(=O)O.CCCC(=O)NC1(CC#N)C23CC4=NN5C(=O)C67C8=C9OC%10C%11=C%12CC(=O)OS(=O)(=O)C1%13OC1C%14CNCC1%15C%13=C(CC(=O)O)N1C%13%16OC(=O)C(C)C%17%18ON=C%19OC(=O)C(OC(=O)CC)(OC(=O)CCC%20(CCOC(=C(CC(=O)O)N(CCC2O4)C3)C%20)C%17(C(=O)O)S%10(=O)=O)C62ON=C3C%12C(=O)N4OC(=O)C6CC%11%13C9C6(COC6CCCOC6)NC(CC(=O)O)=NCCOC69C%10CC%11(CN6C5(C)C(=O)O)C56OC%12%13C5(C(=O)N5C(=C=NOC5(C)C(=O)O2)C(=O)OS(=O)(=O)C(CN2CCCC2)C(=O)O%10)C1(OC(=O)CC)C%12(C4(C(=O)O%19)C%11%16%13)C61C(N)(CO)OC(=O)C1(C%15%18)C81OC(OC(=O)C(C)=C(C)CC(=O)NCCC#N)C(=C(COC(=O)CC)C(=O)O)C1C1(C2CN(C(=C=NO)C(C2=C(C)C(=O)O)C%149N(C)CC)C12CCNCC2)N37.NCCO'